In [21]:
import tkinter as tk
#from PIL import ImageTk, Image
from requests import request
#from io import BytesIO
from IPython.display import Image, HTML
#from IPython.core.display import HTML 
import math

In [2]:
# Initial Dealer Setup
class Dealer:
    def __init__(self):
        self.name = 'Dealer'
        self.cards = []
        self.hidden_card = "https://deckofcardsapi.com/static/img/XX.png"
    
    def add_card(self, card):
        return self.cards.append(card)
    
    def clear_cards(self):
        self.cards = []
        return self.cards
            

In [23]:
# Initial Player Setup

class Player(Dealer):
    def __init__(self):
        self.name = self.set_name()
        self.balance = 1000
        self.cards = []
        self.cards_split = []
        self.bet = 0
    
    def set_name(self):
        name = input('Ingresa tu nombre: ')
        return name
        
    def bet(self):
        print(f"Tu balance actual es de {self.balance}. Ingresa tu apuesta, recuerda que tiene que ser un número entero")
        while True:    
            try:
                bet = int(input('Apuesta: '))
                if bet < self.balance:
                    self.bet = bet
                    self.balance -= bet
                else:
                    print("No puedes apostar más de lo que tienes en tu balance. Intenta una apuesta menor.")
                    return self.bet()
                    break
            except ValueError:
                print("No ingresaste un número intentalo de nuevo.")
                
            
    def add_card(self, card):
        self.cards.append(card)
    
    def clear_cards(self):
        self.cards = []
        return self.cards

In [18]:
#Initial game Setup

class Game:
    def __init__(self):
        self.player = Player()
        self.dealer = Dealer()
        self.deck = self.initialize_deck()
        self.cards_remaining = 0
        self.turn = 'player'
        
    def initialize_deck(self):
        cards = 'https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=6'
        response = request('GET',cards).json()
        return response['deck_id']
    
    def play_hand(self):
        if self.player.balance > 0:
            self.player.bet()
            self.deal_cards()
        
            self.print_game()
            player_values = [card[1] for card in self.player.cards]
            for value in player_values:
                if value
            dealer_values = [card[1] for card in self.player.cards]
        else:
            resp = input('Ya no tienes balance, deseas comenzar un juego nuevo? (s) Sí, (n) No')
            if resp == 's':
                game.__init__()
                         
    
    def draw(self):
        draw = f'https://deckofcardsapi.com/api/deck/{self.deck}/draw/?count=1'
        drawn_card = request('GET', draw).json()
        img = drawn_card['cards'][0]['images']['png']
        value = drawn_card['cards'][0]['value']
        code = drawn_card['cards'][0]['code']
        self.cards_remaining = drawn_card['remaining']
        return [img, value, code]
    
    def deal_cards(self):
        self.player.add_card(self.draw())
        self.dealer.add_card(self.draw())
        self.player.add_card(self.draw())
        self.dealer.add_card(self.draw())   
            
    def hand_total(self, hand):
        hand_values = [card[1] for card in hand]
        total1 = 0
        total2 = 0
        if len(hand_values) == 2 and 'ACE' in hand_values and ('KING' in hand_values or 'QUEEN' in hand_values or 'Jack' in hand_values or '10' in hand_values):
            return '21 - BLACKJACK!'
        for value in hand_values:
            if value == 'ACE':
                total1 += 1
                total2 += 11
            elif value == 'KING' or value == 'QUEEN' or value == 'JACK':
                total1 += 10
                total2 += 10
            else:
                total1 += int(value)
                total2 += int(value)
            return total1, total2
    
    def initial_bj_evaluation(self):
        player_total = self.hand_total(self.player.cards)
        dealer_total = self.hand_total(self.dealer.cards)
        if '21 - BLACKJACK!' in player_total:
            self.player.balance += self.player.bet + math.floor(self.player.bet*3/2)
            return "BLACKJACK! YOU WIN! 3:2", self.play_hand()
        elif '21 - BLACKJACK' in dealer_total:
            self.turn = 'Dealer'
            return "DEALER BLACKJACK - YOU LOSE!", self.print_game(), self.play_hand()
        else:
            self.player_action(self.player.cards)
    
    def bj_evaluation(self, hand):
        hand_total = self.hand_total(hand)
        if '21 - BLACKJACK!' in hand_total:
            self.player.balance += self.player.bet + math.floor(self.player.bet*3/2)
            return "BLACKJACK! YOU WIN! 3:2"
            
    def player_action(self, hand):
        if len(hand) == 2 and hand[0][1] == hand[1][1]:
            action = input('Ingresa tu acción - (s) Split, (d) Double Down, (n) Hit - New Card, (h) Hold or Stand\n')
            if action == 's':
            index = self.player.cards.index(hand)
            original1 = self.player.cards[index][0]
            original2 = self.player.cards[index][1]
            self.player.cards[index] = [original1, self.draw()]
            self.player.cards.append([original2, self.draw()])
            player_total = self.hand_total(self.player.cards)
            player_split_total = self.hand_total(self.player.cards_split)
    
    def print_game(self):
        dealer_cards = ""
        player_cards = ""
        if self.turn == "player":
            dealer_cards = f"""
            <td><img src={self.dealer.hidden_card} style="width: 60px;"/></td>
            <td><img src={self.dealer.cards[1][0]} style="width: 60px;"/></td>
            """
        else:
            for card in self.dealer.cards:
                dealer_cards += f"""<td><img src={card[0]} style="width: 60px;"/></td>"""
        
        for card in self.player.cards:
            player_cards += f"""<td><img src={card[0]} style="width: 60px;"/></td>"""
        
        return HTML(f""" 
                <table>
                <tr><h1> Dealer Hand </h1></tr>
                <tr>
                {dealer_cards}
                </tr>
                </table>
                
                <table>
                <tr><h1> {self.player.name}'s Hand </h1></tr>
                <tr>
                {player_cards}
                </tr>
                </table>
        """)

In [19]:
#Instantiating Player
game = Game()
game.player.__dict__

Ingresa tu nombre: Rob


{'name': 'Rob', 'balance': 1000, 'cards': []}

In [20]:
# Begin Hand
game.play_hand()

Tu balance actual es de 1000. Ingresa tu apuesta, recuerda que tiene que ser un número entero
Apuesta: 100
['5', '2']
['4', '9']


In [ ]:
game.__dict__

In [ ]:
atributes = vars(dealer)
print(atributes)

In [ ]:
game.print_cards()